In [1]:
!pip install tensorflow==2.0.0

In [2]:
from google.colab import drive
import sys
drive.mount('/content/gdrive/',force_remount=True)

o_root = r"/content/gdrive/My Drive/Puc/Projeto Final"
sys.path.append(f'{o_root}/Code/source')

Mounted at /content/gdrive/


In [0]:
import os
if not os.path.isdir("data"):
  print("No data folder. Building all")
  !7za x '/content/gdrive/My Drive/Puc/Projeto Final/Datasets/tuned/train_sliding.7z' -odata -mmt4 -y

In [0]:
from training.train_mentions import slice_and_split_input, input_fn_params
import os
from tqdm import tqdm

DATA_FOLDER=f"{o_root}/Datasets/separation_40"
os.environ["BERT_DB_PATH"]=f"data/sliding"

NUM_CPU_READ = 2
BUFFER_SIZE = 100
EPOCHS = 100
VALIDATION_STEPS = 50
VALIDATIONS_PER_EPOCH = 15
BATCH_SIZE = 10
REDUCTIONS_STEPS=5


train, valid = slice_and_split_input(DATA_FOLDER)
params = input_fn_params(BATCH_SIZE, 0, BUFFER_SIZE, False)
params["num_cpu_threads"] = 10
for batch in tqdm(train(params), desc="Train"):
  for raw_record in batch:
    pass

Using 2080 files for train and 694 for validation


Train: 34it [05:18,  9.13s/it]

In [0]:
import tensorflow as tf
import os
import numpy as np
from tqdm import tqdm
import shutil
from training.train_mentions import MAX_NUM_MENTIONS, MAX_NUM_PREDICTIONS, INPUT_EMBEDDINGS_SIZE, slice_and_split_input


def create_float_feature(values):
    feature = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
    return feature


def create_int_feature(values):
    f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
    return f


name_to_features = {
    "input_embeddings":
        tf.io.FixedLenFeature(MAX_NUM_MENTIONS * INPUT_EMBEDDINGS_SIZE, tf.float32),
    "input_mask":
        tf.io.FixedLenFeature(MAX_NUM_MENTIONS, tf.int64),
    "labels":
        tf.io.FixedLenFeature(MAX_NUM_PREDICTIONS, tf.int64),
    "output_mask":
        tf.io.FixedLenFeature(MAX_NUM_PREDICTIONS, tf.int64),
    "cumsum":
        tf.io.FixedLenFeature(MAX_NUM_PREDICTIONS * INPUT_EMBEDDINGS_SIZE * 5, tf.float32),
}


def to_feature(tensor):
    as_numpy = tensor.numpy()
    if as_numpy.dtype == np.float32:
        return create_float_feature(as_numpy)
    return create_int_feature(as_numpy)


def check(out_dir, ds):
    counter = 0
    for batch in tqdm(ds):
        new_file = f"{out_dir}/{counter}.tsv"
        counter += 1
        with tf.io.TFRecordWriter(new_file) as writer:
            for raw_record in batch:
                as_dict = tf.io.parse_single_example(raw_record, name_to_features)
                features = {x: to_feature(as_dict[x]) for x in as_dict}
                example = tf.train.Example(features=tf.train.Features(feature=features))
                writer.write(example.SerializeToString())


main_root = f"{o_root}/tsv/separation_40"
work_dir = "work"

os.makedirs(f"{work_dir}/data_in", exist_ok=True)
os.makedirs(f"{work_dir}/data_out", exist_ok=True)
slice_and_split_input

for root, dirs, current_files in os.walk(main_root):
    for file_name in current_files:
        print(f"{file_name}")
        if "_" not in file_name:
            print("Ignoring")
            continue
        shutil.copy(f"{root}/{file_name}", f"{work_dir}/data_in")
        ds = tf.data.TFRecordDataset(filenames=[f"{work_dir}/data_in/{file_name}"])
        ds = ds.batch(10)
        check(f"{work_dir}/data_out", ds)

        subfolders = file_name.split("_")[:3]
        out_dir = os.path.join(main_root, *subfolders, file_name.replace(".tsv", ""))
        os.makedirs(out_dir, exist_ok=True)
        
        print(f"Moving files to {out_dir}")
        for src in os.listdir(f"{work_dir}/data_out"):
            shutil.move(f"{work_dir}/data_out/{src}", out_dir)
        os.remove(f"{work_dir}/data_in/{file_name}")
        os.remove(f"{root}/{file_name}")